# [Mathematics: Linear Algebra] Manipulating images with matrix operations
Goal: Perform some basic transformations on an image using vector/matrix operations.

First, we load an image using the Python Image Processing Library. Note that even though we'll reimplement most of the library's functionality (crop, resize, scale, merge methods), we're not reimplementing image reading, as that would mean interpreting bytes based on the image spec (JPEG/PNG/whatever), and that isn't our focus.

In [74]:
import math
import numpy as np
from PIL import Image

image = Image.open('fixtures/koala.jpg')

# This works because the Image class implements .__array_interface__()
matrix = np.array(image)

# It returns a 3D array (Would have been nicer if it returned a 2D array of 3-tuples)
print(matrix.shape)
print(matrix[0][0])


(768, 1024, 3)
[101  90  58]
<class 'numpy.ndarray'>


## Understanding the image
The image is 1024 by 768, in RGB mode. Its matrix has the shape (768, 1024, 3). So this is a 2-dimensional matrix (768 * 1024) with 3 items each, representing the RGB value of each pixel. `matrix[i][j]` will be `[red_value  green_value  blue_value]`.

A key to the matrix manipulations is treating this as a set of position vectors that we manipulate independently.

## Challenge 1: Size manipulation - Crop
Let's say we want to crop the image to a central portion half the original area. TO do this without distortion, we must maintain the shape (ie the aspect ratio of 4:3, or 1024:768).

Area of image = 1024 * 768 = 786 432 pixels².

Desired area = 393 216 pixels².

Length and width to give us this area while maintaining the aspect ratio: 4p * 3p = 12p² = 393216 => p = 181.02.

Thus, l = 4(181.02) and w 3(181.02) => 724 by 543 (round to 544 to be an even number).

If we call the 768 dimension x and the 1024 dimension y, this means we will keep the inner 544 in x (remove 112 at the start and end) and the inner 724 in y (remove 150 at the start and end).

In [3]:
cropped_half_area = matrix[112:-112, 150:-150]

Image.fromarray(cropped_half_area).show()

We can also instead crop to a specific aspect ratio, say 1:1 (as for a profile picture). The resulting dimensions will then be 768 by 768, since we can only crop to the smaller dimension. If we still want to crop outwards from the centre, this means we'll take the full 768 in x and keep 768 from y (remove 128 from each end).

In [4]:
cropped_1_1 = matrix[:, 128:-128]
Image.fromarray(cropped_1_1).show()

Next challenge: cropping to a circular shape. To do this, we'll find all pixels outside the circle and zero them. The radius of our circle will be 768/2 = 384. We can go over each cell and zero it if its distance from the centre is greater than 384.

In [25]:
x, y, _ = matrix.shape
diameter = min(x, y)
radius = diameter / 2
centre = ((x + 1) / 2, (y + 1) / 2)

# X is a 768 x 1 column vector, represented by 768 arrays of 1 element each
# Y is a 1 x 1024 row vector, represented by 1 array of 1024 items
X, Y = np.ogrid[:x, :y]
a = (X - centre[0]) ** 2 
b = (Y - centre[1]) ** 2
z = a + b
distance = np.sqrt((X - centre[0]) ** 2 + (Y - centre[1]) ** 2)
mask = distance > radius

cropped_circular = np.array(matrix)
cropped_circular[mask] = [0, 0, 0]  # We could do ~mask to invert the crop

Image.fromarray(cropped_circular).show()

The initial implementation of this looped over each position vector, calculated the distance and then unset it if it failed. This was not performant (took 3 seconds). NumPy gives us more powerful ways to do this (pushing the computations to C):
- With operators and broadcasting, comparing each element of the distance matrix to the radius is nearly instant.
- With masking, we can easily modify a whole set of cells.


## Challenge 2: Size manipulation - Resize
How do we reduce the size of this image without changing its other dimensions? We need to keep its original 4:3 aspect ratio, which is 4:3 (1024:768).

Supposing we want to reduce it to 600x400. This is a reduction of 1.28. So we must replace every 1.28 pixels in the original image by 1 pixel, or every 64 pixels must be replaced by 50.

We can do this with matrix operations. Scaling a position vector by k is equivalent to pre-multiplying by $ \begin{psmallmatrix} k & 0 \\ 0 & k \end{psmallmatrix} $, ie:

$$
\begin{pmatrix} k & 0 \\ 0 & k \end{pmatrix} \cdot \begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} kx \\ ky \end{pmatrix}
$$

In [35]:
scaling_factor = 600 / 1024
scaling_matrix = np.array([[scaling_factor, 0], [0, scaling_factor]])

# First, allocate output matrix by applying the scaling matrix to the original dimensions
new_dimensions = np.matmul(scaling_matrix, matrix.shape[0:2]).astype(int)
scaled = np.empty((*new_dimensions, matrix.shape[2]), dtype=matrix.dtype)

for i1 in range(matrix.shape[0]):
    for j1 in range(matrix.shape[1]):
        i2, j2 = np.matmul(scaling_matrix, [i1, j1])
        scaled[int(i2), int(j2)] = matrix[i1, j1]

Image.fromarray(scaled).show()


This implementation has some limitations:
1. It loops over each pixel. There's probably a more efficient way to do this in NumPy, but I couldn't figure it out.
2. The scaling is not very precise. Scaling can lead to non-integral coordinates, and I'm simply casting them to int, but there's no guarantee we won't have some coordinates to which nothing maps. Even if we do, there's no guarantee that it's the "right" pixel; if we must precisely replace every 1.28 pixels by 1, we should ideally find the two neighbouring pixels to be merged and decide on the appropriate replacement.

   Additionally, scaling up (eg resizing to 1920x1080) will lead to some pixels being blank (since we now have more pixels). For precision, we should look at those blank pixels and decide how to fill them from neighbouring ones. I heard of a technique called bilinear interpolation for this. 


## Challenge 3: Geometrical manipulation - Rotate
For rotation, we can once again multiply a matrix, in this case, 
$$
\begin{pmatrix} \cos θ & -\sin θ \\ \sin θ & \cos θ \end{pmatrix}
$$

Proof: if a vector (x, y) originally at angle α and with magnitude of p, is rotated through α+θ, its new coordinates will be $ (p \cos(α+θ), p \sin(α+θ)) = (p[\cos(α)\cos(θ) - \sin(α)\sin(θ)], p[\sin(α)\cos(θ) + \cos(α)\sin(θ)]) = (x \cos θ - y \sin θ, y \cos θ + x \sin θ) $.

Let's rotate by 270° ($ \frac{3}{2}π $ radians).


In [43]:
theta = 3 * np.pi / 2
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]).astype(int)

# First, allocate output matrix by applying the scaling matrix to the original dimensions
# Use abs to get rid of the negative dimensions as we only need the absolute length
new_dimensions = np.abs(np.matmul(rotation_matrix, matrix.shape[0:2]).astype(int))
rotated = np.empty((*new_dimensions, matrix.shape[2]), dtype=matrix.dtype)

for i1 in range(matrix.shape[0]):
    for j1 in range(matrix.shape[1]):
        i2, j2 = np.matmul(rotation_matrix, [i1, j1])
        rotated[i2][j2] = matrix[i1, j1]

Image.fromarray(rotated).show()



## Challenge 4: Geometrical manipulation - Reflect
To reflect geometrically around the y-axis, the mapping is $ f: x \rightarrow -x $, where x is the coordinate (eg 1024 or 768). So we only need to invert the indexes.
 

In [6]:
reflected_in_y = matrix[:, ::-1]
reflected_in_x = matrix[::-1]
reflected_in_xy = matrix[::-1, ::-1]
Image.fromarray(reflected_in_xy).show()


## Challenge 5: Colour manipulation - Filter
We can also manipulate the image's appearance by adjusting the RGB values. For instance, let's halve all blues.

In [71]:
filtered = np.copy(matrix)
# We could also just do filtered[:,:,2] *= 0.5, but this fails because of the casting (float to int)
filtered[:, :, 2] = np.multiply(0.5, filtered[:, :, 2], casting='unsafe')

Image.fromarray(filtered).show()


## Conclusion
We've used some basic linear algebra to demonstrate the basic principles of image manipulations. The keys:
- treating the pixel matrix indexes as a set of position vectors, which we could then manipulate to transform the image
- manipulating RGB values to apply filters 